# Кредитный скоринг

# Кредит

* Заявитель заполняет анкету
* Банк принимает решение
* Клиент пользуется кредитом
* Клиент возвращает долг или не возвращает

# Специфика задачи кредитного скоринга

* Классы “good” и “bad” неразделимы
* Предсказание поведения человека
* Долгосрочное прогнозирование
* Большая цена ошибки

# История кредитного скоринга



# Дополнительные сервисы

* Автоматизация процесса принятия решения
* Решение принимается за секунды
* Используется множество источников информации
* Сформирован рынок продажи информации для банков

# Примеры задач

* Application scoring
* Behavior scoring
* Collection scoring
* Marketing
* Churn analysis

# Формализация задачи

* Конечная выборка размера n.
* $X_i$ – вектор признаков.
* $y_i$ – бинарная целевая переменная.
* $(X_i, y_i)$ - независимые одинаково распределенные наблюдения.
* Необходимо найти функцию $S(X) ≃ \mathbb{P}(y = 1| X)$

# Метрики качества

* MSE (Mean square error)
* logloss (log-Likelihood)
* AUC (Area under the ROC curve)
* IV (Information Value)

# Среднеквадратичное отклонение
* MSE (Mean square error)  
  $\mathbb{E}[(y - S(X))^2]$

* Оптимальное значение  
  $S(X) = \mathbb{P}(y = 1| X)$

* Оценка  
  $\frac{1}{n}\sum_{i=1}^n[(y_i - S(X_i))^2]$

# Правдоподобие

* logloss (log-Likelihood)  
  $-LL = -\mathbb{E}[y \ln(S(X)) + (1-y)\ln(1 - S(X))]$

* Оптимальное значение  
  $S(X) = \mathbb{P}(y = 1| X)$

* Оценка  
  $\frac{1}{n}\sum_{i=1}^n[y_i \ln(S(X_i)) + (1-y_i)\ln(1 - S(X_i))]$